In [1]:
#@title setup, static and constant data

# Mount Google Drive
import os
from google.colab import drive
gdrive = '/content/drive/'
gdriveprefix = gdrive + 'My Drive/Colab Notebooks/'
drive.mount(gdrive, force_remount=True)

import numpy as np
import pandas as pd
import urllib.request

# useful constants
tab = "\t";
newline = "\n";

# sitelist data locations
data_prefix ='https://raw.githubusercontent.com/bdekoz/mozilla-data-lcp/main/';
sitebase = 'CrUX_global_200_10M-2023-12';
sitelist = sitebase + '.txt.ping';
sitefile = data_prefix + "sitelists/CrUX.2023-12/" + sitelist;

# sitelist currently reachable
errfile = gdriveprefix + sitebase + ".error.txt";
okfile = gdriveprefix + sitebase + ".pass.txt";




Mounted at /content/drive/


In [2]:
#@title percentage(numerator, denom)

def percentage(part, whole):
  Percentage = 100 * float(part)/float(whole)
  return str(int(round(Percentage,0))) + '%'




In [3]:
#@title origin_check_readable(origin, log) / origin_contains(origin, match)

import requests

# check origin to see if it can be read
def origin_check_readable(origin, logfile):
  try:
    r = requests.get(origin);
  except:
    logfile.write(origin + newline);


def origin_contains(origin, match, logfile):
  matchp = False
  r = requests.get(origin);
  if match in r.text:
    matchp = True
  return matchp

In [ ]:
#@title check sitelist

with urllib.request.urlopen(sitefile) as response:
  print("found: " + sitefile);

  errlog = open(errfile, "w");
  passlog = open(okfile, "w");
  print(errfile);
  print(okfile);

  for line in response.readlines():
    origin = line.decode("ascii").strip(newline); # utf-8, ascii
    try:
      origin_check_readable(origin, errlog);
      passlog.write(origin + newline);
    except:
      continue;

  errlog.close();
  passlog.close();

found: https://raw.githubusercontent.com/bdekoz/mozilla-data-lcp/main/sitelists/CrUX_global_200_10M-2023-12.txt.ping
/content/drive/My Drive/Colab Notebooks/CrUX_global_200_10M-2023-12.error.txt
/content/drive/My Drive/Colab Notebooks/CrUX_global_200_10M-2023-12.pass.txt


In [ ]:
#@title create_content_setasides(sitefile, tag, match)

# Open the URL and read the contents, establish sitelist size.
siten=0;
with urllib.request.urlopen(sitefile) as response:
  siten = len(response.readlines())
print("sites: " + str(siten))


# Web Content string fragments
# <link rel="preconnect" or <link rel=preconnect
# <link rel="dns-prefetch"
# https://developer.mozilla.org/en-US/docs/Web/Performance/Speculative_loading
# https://developer.mozilla.org/en-US/docs/Web/Performance/dns-prefetch
# https://fetch.spec.whatwg.org/#concept-request-destination
wc1='rel="dns-prefetch"';
wc1a='rel=dns-prefetch';

wc2='rel=preconnect';
wc2a='rel="preconnect"';

wc3='rel="preload"';
wc3a='rel=preload';

wc4='rel="prefetch"';
wc4a='rel=prefetch';

wc5='rel="prerender"';
wc5a='rel=prerender';

wc5='rel="dictionary"';
wc5a='rel=dictionary';


def create_content_traits_1(sitefile, tag, matchstr):
  log = open(gdriveprefix + tag + ".error.log", "w");
  df = pd.DataFrame(columns=[tag], index=range(siten))

  # Scan and store data for lookups.
  with urllib.request.urlopen(sitefile) as response:
    linen = 0
    for line in response.readlines():
      origin = line.decode("ascii").strip(newline); # utf-8, ascii

      try:
        matchp = False
        r = requests.get(origin);
        if matchstr in r.text:
          matchp = True
        print(str(linen) + tab + str(matchp));
        df.iloc[linen, 0] = matchp;
      except:
        log.write(str(linen) + ',' + origin + newline);
        df.iloc[linen, 0] = np.NaN;
        continue;
      finally:
        linen += 1;

  print("finished");
  log.close();
  df.to_csv(gdriveprefix + tag + ".csv");


def create_content_traits_2(sitefile, tag, matchstr1, matchstr2):
  log = open(gdriveprefix + tag + ".error.log", "w");
  df = pd.DataFrame(columns=[tag], index=range(siten))

  # Scan and store data for lookups.
  with urllib.request.urlopen(sitefile) as response:
    linen = 0
    for line in response.readlines():
      origin = line.decode("ascii").strip(newline); # utf-8, ascii

      try:
        matchp = False
        r = requests.get(origin);
        if matchstr1 in r.text:
          matchp = True
        if matchstr2 in r.text:
          matchp = True
        print(str(linen) + tab + str(matchp));
        df.iloc[linen, 0] = matchp;
      except:
        log.write(str(linen) + ',' + origin + newline);
        df.iloc[linen, 0] = np.NaN;
        continue;
      finally:
        linen += 1;

  print("finished");
  log.close();
  df.to_csv(gdriveprefix + tag + ".csv");


#create_content_traits_1(sitefile, "wc1", wc1);
#create_content_traits_1(sitefile, "wc2", wc2);
#create_content_traits_1(sitefile, "wc3", wc3);
#create_content_traits_1(sitefile, "wc4", wc4);
#create_content_traits_1(sitefile, "wc5", wc5);

#create_content_traits_2(sitefile, "dns-prefetch", wc1, wc1a);
create_content_traits_2(sitefile, "preconnect", wc2, wc2a);
create_content_traits_2(sitefile, "preload", wc3, wc3a);
create_content_traits_2(sitefile, "prerender", wc4, wc4a);
create_content_traits_2(sitefile, "dictionary", wc5, wc5a);


sites: 150
0	False
1	False
2	False
3	False
4	False
5	False
6	False
7	False
8	False
9	False
11	False
12	False
13	False
14	False
15	False
16	False
17	False
18	False
20	False
21	False
22	False
23	False
24	False
25	False
26	False
27	False
29	False
30	False
31	True
32	False
33	False
34	False
35	False
36	False
37	False
38	False
39	False
40	False
41	False
42	False
43	False
44	False
45	False
46	False
47	False
48	False
49	False
50	False
51	False
53	False
55	False
56	False
57	False
58	False
59	False
60	False
61	False
62	False
63	False
64	False
66	True
67	False
68	False
69	False
70	False
71	False
72	False
73	False
74	False
75	False
76	False
77	False
78	False
79	False
80	False
81	False
82	False
84	False
85	False
86	False
87	False
88	False
89	False
90	False
91	False
92	False
93	False
94	False
95	False
96	False
97	False
98	False
99	False
100	False
101	False
102	False
103	False
104	False
105	False
106	False
107	False
109	False
110	False
111	False
112	True
113	False
114	False
115	False
116	False
117	F